## PLST - Rebalancing Data for July 2024

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase, MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
date = '2024-07-01'
min_mcap =  0
min_weight = 0.005
max_weight = 0.15
circ_supply_threshold = 0
liveness_threshold = 30 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["liquid-staking-tokens","liquid-restaking-tokens"]
index_address = '0x8b16626d12E7ef3B41712bB8Ff88B8E4e04c165B'

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                2,
                                'arbitrum-one',
                                index_address,
                                'plsx_benchmark_data',
                                'plsx_liquidities',
                                min_mcap,
                                min_weight,
                                max_weight,
                                50000,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)
print(max_slippage)

-0.04


### Token inclusion criteria

1. **Project and token characteristics**
    1. The token should be listed on CoinGecko with pricing data spanning at least 1 month prior to the date of inclusion in the index.
    1. No rebasing tokens, the staking yield should accrue to the token's price.
    1. The token should be 100% collateralised by the underlying assets.
   <br>
1. **Pricing Requirements**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange on a supported chain.
    1. The price should experience no more than 5% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety.
1. **Weighting Requirements**
    1. Each token is weighted according to its market cap.
    1. The maximum weight any one token can have is 15%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the maximum weight the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    weight_by=["yield",methodology_class.get_yield_data],
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[['staked-ether'],['wrapped-steth']],  
    remove_category_assets=['frax-ether','ankreth'],
    platforms_to_add = [{'id':'kelp-dao-restaked-eth','blockchain':'arbitrum-one','address':'0x4186BFC76E2E237523CBC30FD220FE055156b41F'}],
    platforms_to_remove=['ethereum'],   
    values_to_update=[{'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"},
                      {'id':'rocket-pool-eth','category':'symbol','value':"rETH"},
                      {'id':'wrapped-steth','category':'symbol','value':"wstETH"},
                      {'id':'staked-frax-ether','category':'symbol','value':"sfrxETH"},
                      {'id':'stader-ethx','category':'symbol','value':"ETHx"},
                      {'id':'stakewise-v3-oseth','category':'symbol','value':"osETH"},
                      {'id':'restaked-swell-eth','category':'circulating_supply','value':114563.7350152765},
                      {'id':'restaked-swell-eth','category':'total_supply','value':114563.7350152765},
                      {'id':'restaked-swell-eth','category':'market_cap','value':114563.7350152765 * 3050},
                      {'id':'restaked-swell-eth','category':'symbol','value':'rswETH'},
                      {'id':'renzo-restaked-eth','category':'circulating_supply','value':990000},
                      {'id':'renzo-restaked-eth','category':'total_supply','value':990000},
                      {'id':'renzo-restaked-eth','category':'market_cap','value':990000*3060},
                      {'id':'renzo-restaked-eth','category':'symbol','value':'ezETH'},
                      ],
    onchain_oracles=['coinbase-wrapped-staked-eth','lido-staked-matic','sweth','kelp-dao-restaked-eth','stader-maticx','wrapped-eeth'],
    )
liquidity

/Users/olivermehr/Documents/GitHub/rebalancing-data/PLSX/../methodology.py:166: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.category_data = pd.concat(dataframe_list, axis=0)


Excluding lista, pricing data available only for 28 < 30 days
Table created successfully
Table created successfully
Manually calculating slippage for wrapped-steth on ethereum
Manually calculating slippage for wrapped-steth on optimistic-ethereum
zksync not supported
Manually calculating slippage for wrapped-steth on arbitrum-one
Manually calculating slippage for wrapped-steth on polygon-pos
Manually calculating slippage for wrapped-steth on base
linea not supported
mantle not supported
Manually calculating slippage for renzo-restaked-eth on ethereum
Manually calculating slippage for renzo-restaked-eth on binance-smart-chain
Manually calculating slippage for renzo-restaked-eth on base
Manually calculating slippage for renzo-restaked-eth on arbitrum-one
linea not supported
mode not supported
blast not supported
Manually calculating slippage for rocket-pool-eth on ethereum
Manually calculating slippage for rocket-pool-eth on optimistic-ethereum
Manually calculating slippage for rocket-po

,arbitrum-one,avalanche,base,binance-smart-chain,fantom,optimistic-ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain
id,,,,,,,,,,,
wrapped-eeth,-0.000116,NaN,NaN,NaN,NaN,NaN,NaN,arbitrum-one,-0.000116,-0.000116,arbitrum-one
coinbase-wrapped-staked-eth,-0.003906,NaN,-0.000170,NaN,NaN,NaN,-0.979481,arbitrum-one,-0.000170,-0.003906,base
renzo-restaked-eth,-0.000235,NaN,-0.000260,-0.636493,NaN,NaN,NaN,arbitrum-one,-0.000235,-0.000235,arbitrum-one
kelp-dao-restaked-eth,-0.000278,NaN,NaN,NaN,NaN,NaN,NaN,arbitrum-one,-0.000278,-0.000278,arbitrum-one
wrapped-steth,-0.000616,NaN,-0.001115,NaN,NaN,-0.033732,-0.008391,arbitrum-one,-0.000616,-0.000616,arbitrum-one
rocket-pool-eth,-0.000744,NaN,-0.001484,NaN,NaN,-0.330526,-0.936440,arbitrum-one,-0.000744,-0.000744,arbitrum-one
benqi-liquid-staked-avax,NaN,-0.003722,NaN,NaN,NaN,NaN,NaN,avalanche,-0.003722,-0.003722,avalanche
stader-maticx,NaN,NaN,NaN,NaN,NaN,NaN,-0.020641,polygon-pos,-0.020641,-0.020641,polygon-pos
binance-eth,NaN,NaN,NaN,-0.037619,NaN,NaN,NaN,binance-smart-chain,-0.037619,-0.037619,binance-smart-chain


In [4]:
methodology_class.show_results().sort_values('weight',ascending=False)

,symbol,market_cap,weight,weight_converted,address,blockchain
benqi-liquid-staked-avax,sAVAX,2.309304e+08,0.150000,9831,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
kelp-dao-restaked-eth,RSETH,8.758623e+08,0.147607,9674,0x4186BFC76E2E237523CBC30FD220FE055156b41F,arbitrum-one
stader-maticx,MATICX,8.538470e+07,0.147607,9674,0xfa68fb4628dff1028cfec22b4162fccd0d45efb6,polygon-pos
lido-staked-matic,STMATIC,0.000000e+00,0.121576,7967,0x3a58a54c066fdc0f2d55fc9c89f0415c92ebf3c4,polygon-pos
coinbase-wrapped-staked-eth,CBETH,7.425524e+08,0.092641,6071,0x1debd73e752beaf79865fd6446b0c970eae7732f,arbitrum-one
renzo-restaked-eth,ezETH,3.029400e+09,0.091539,5999,0x2416092f143378750bb29b79ed961ab195cceea5,arbitrum-one
wrapped-steth,wstETH,3.369758e+10,0.087735,5749,0x5979d7b546e38e414f7e9822514be443a4800529,arbitrum-one
wrapped-eeth,WEETH,5.107634e+09,0.085818,5624,0x35751007a407ca6feffe80b3cb397736d2cf4dbe,arbitrum-one
rocket-pool-eth,rETH,1.937154e+09,0.075477,4946,0xec70dcb4a1efa46b8f2d97c310c9c4790ba5ffa8,arbitrum-one


In [5]:
methodology_class.output_for_contract()

{43114: {'0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be': 9831}, 42161: {'0x4186BFC76E2E237523CBC30FD220FE055156b41F': 9674, '0x1debd73e752beaf79865fd6446b0c970eae7732f': 6071, '0x2416092f143378750bb29b79ed961ab195cceea5': 5999, '0x5979d7b546e38e414f7e9822514be443a4800529': 5749, '0x35751007a407ca6feffe80b3cb397736d2cf4dbe': 5624, '0xec70dcb4a1efa46b8f2d97c310c9c4790ba5ffa8': 4946}, 137: {'0xfa68fb4628dff1028cfec22b4162fccd0d45efb6': 9674, '0x3a58a54c066fdc0f2d55fc9c89f0415c92ebf3c4': 7967}}
